In [ ]:
import tensorflow as tf
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [ ]:
x = tf.Variable(3, name="x")
z = tf.Variable(2, name="z")
f = x * z + 2


In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    result = f.eval()
    print (result)

In [ ]:
#DNN

In [ ]:
data = load_boston()
X_train, X_test, y_train, y_test =  train_test_split(data.data, data.target)

In [ ]:
ss =  StandardScaler()
X_train = ss.fit_transform(X_train)
X_test =  ss.fit_transform(X_test)

In [ ]:
X_train.shape

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, shape=(None,13), name='X')
y = tf.placeholder(dtype=tf.float32, shape=(None), name='y')

h1= tf.layers.dense(X, 13, name='hidden1', activation=tf.nn.relu)
h2= tf.layers.dense(h1, 5, name='hidden2', activation=tf.nn.relu)
y_hat = tf.layers.dense(h2, 1, name='y_hat',activation=None)

loss = tf.sqrt(tf.reduce_mean(tf.square(y_hat - y)))

gd = tf.train.AdamOptimizer(.01)
traininig_op = gd.minimize(loss)

In [ ]:
kf = KFold(5, random_state=3)
batch_size_indeces = [test for _, test in kf.split(X_train,y_train)]

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(100):
        for batch in batch_size_indeces:
            X_batch = X_train[batch]
            y_batch = y_train[batch]
            sess.run(traininig_op, feed_dict={X: X_batch, y:y_batch})
        train_loss = sess.run(loss,feed_dict={X:X_train, y:y_train})
        test_loss = sess.run(loss,feed_dict={X:X_test, y:y_test})
    

        print("epoch", epoch, 'train loss', train_loss,"test loss", test_loss)
        if epoch % 10 == 10:
            saver.save(sess, './housing.ckpt')
    pred = sess.run(y_hat, feed_dict={X:X_test,y:y_test})
print (metrics.r2_score(y_test, pred))
            
        
    
    

In [ ]:
with tf.Session() as sess:
    saver.restore(sess,'housing.ckpt')
    pred=sess.run(y_hat, feed_dict={X:X_test, y:y_test})
    print (pred[0])